In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_moda = pd.read_csv("../Credit-Card-Prediction/Base Moda.csv", sep=',')
df_ANS = pd.read_csv('../Credit-Card-Prediction/Base AnS.csv', sep=',')

In [23]:
df_moda.columns

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'STATUS', 'Estado'],
      dtype='object')

In [22]:
#print (len(set(df_ANS['ID']).intersection(set(df_moda['ID']))))
X_cla1 =df_moda.loc[:,["CODE_GENDER","FLAG_OWN_CAR", 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'CNT_FAM_MEMBERS']]

In [24]:
Y_cla =df_moda.loc[:,"Estado"]

In [4]:
def createdummies(df,varname):
    dummy = pd.get_dummies(df[varname], prefix = varname)
    df = df.drop(varname, axis = 1)
    df = pd.concat([df, dummy], axis = 1)
    return df 

In [25]:
X_cla =createdummies(X_cla1,["CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE",
                           "NAME_FAMILY_STATUS","NAME_HOUSING_TYPE"])
X_cla.dtypes

CNT_CHILDREN                                           int64
AMT_INCOME_TOTAL                                     float64
DAYS_BIRTH                                             int64
DAYS_EMPLOYED                                          int64
FLAG_MOBIL                                             int64
FLAG_WORK_PHONE                                        int64
FLAG_PHONE                                             int64
FLAG_EMAIL                                             int64
CNT_FAM_MEMBERS                                      float64
CODE_GENDER_F                                          uint8
CODE_GENDER_M                                          uint8
FLAG_OWN_CAR_N                                         uint8
FLAG_OWN_CAR_Y                                         uint8
FLAG_OWN_REALTY_N                                      uint8
FLAG_OWN_REALTY_Y                                      uint8
NAME_INCOME_TYPE_Commercial associate                  uint8
NAME_INCOME_TYPE_Pension

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

In [8]:
# Logistic Regression Classification
from sklearn.linear_model import LogisticRegression

kfold = KFold(n_splits=10)
model = LogisticRegression(solver= 'lbfgs', max_iter=1000)
results = cross_val_score(model, X_cla, Y_cla, cv= kfold)

print(f"Accracy:: {results.mean()*100:,.2f}% ({results.std()*100:,.2f}%)")

Accracy:: 52.27% (1.69%)


In [ ]:
# LDA Classification
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

kfold = KFold(n_splits=10, random_state=7)
model = LinearDiscriminantAnalysis()
results = cross_val_score(model, X_cla, Y_cla, cv= kfold)

print(f"Accracy:: {results.mean()*100:,.2f}% ({results.std()*100:,.2f}%)")

In [ ]:
# KNN Classification
from sklearn.neighbors import KNeighborsClassifier

kfold = KFold(n_splits=10, random_state=7)
model = KNeighborsClassifier(n_neighbors= 3 ,algorithm='kd_tree' )
results = cross_val_score(model, X_cla, Y_cla, cv= kfold)

print(f"Accracy:: {results.mean()*100:,.2f}% ({results.std()*100:,.2f}%)")

In [ ]:
# Gaussian Naive Bayes Classification
from sklearn.naive_bayes import GaussianNB

kfold = KFold(n_splits=10, random_state=7)
model = GaussianNB()
results = cross_val_score(model, X_cla, Y_cla, cv= kfold)

print(f"Accracy:: {results.mean()*100:,.2f}% ({results.std()*100:,.2f}%)")

In [ ]:
# CART Classification
from sklearn.tree import DecisionTreeClassifier

kfold = KFold(n_splits=10, random_state=7)
model = DecisionTreeClassifier(criterion='entropy')#, splitter='random')
results = cross_val_score(model, X_cla, Y_cla, cv= kfold)

print(f"Accracy:: {results.mean()*100:,.2f}% ({results.std()*100:,.2f}%)")

In [ ]:
# SVM Classification: Crea rectas - hiperplanos en el espacio
from sklearn.svm import SVC

kfold = KFold(n_splits=10, random_state=7)
model = SVC(kernel='poly',  degree=4)
results = cross_val_score(model, X_cla, Y_cla, cv= kfold)

print(f"Accracy:: {results.mean()*100:,.2f}% ({results.std()*100:,.2f}%)")

In [9]:
import lazypredict
import tqdm
import xgboost
import catboost
import lightgbm
import pytest

In [26]:
from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X = X_cla
y= Y_cla
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.3,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 93%|█████████▎| 27/29 [00:10<00:00,  2.50it/s]

[21:18:14] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 29/29 [00:11<00:00,  2.52it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
DecisionTreeClassifier,0.53,0.52,0.52,0.52,0.05
ExtraTreesClassifier,0.53,0.51,0.51,0.51,0.55
SGDClassifier,0.53,0.51,0.51,0.51,0.04
AdaBoostClassifier,0.55,0.51,0.51,0.44,0.22
LGBMClassifier,0.54,0.51,0.51,0.50,0.38
LabelPropagation,0.52,0.51,0.51,0.51,0.72
LabelSpreading,0.52,0.51,0.51,0.51,1.18
BaggingClassifier,0.53,0.50,0.50,0.51,0.23
QuadraticDiscriminantAnalysis,0.47,0.50,0.50,0.40,0.04
